## FineTuning LLM with Model-As-Service

This sample shows how use create a standalone FineTuning job to fine tune a model to summarize a dialog between 2 people using samsum dataset.

#### Training data
We will use the [samsum](https://huggingface.co/datasets/samsum) dataset. This dataset is intended to summarize dialogues between 2 people. with this notebook we will summarize the dialogues and calculate bleu and rouge scores for the summarized text vs provided ground_truth summaries

#### Model
We will use the `llama-2-7b` model to show how user can finetune a model for text-generation task. If you opened this notebook from a specific model card, remember to replace the specific model name. Optionally, if you need to fine tune a model that is available on HuggingFace, but not available in `azureml` system registry, to do so [import](https://github.com/Azure/azureml-examples/blob/main/sdk/python/foundation-models/system/import/import_model_into_registry.ipynb) the model.

#### Outline
1. Setup pre-requisites
2. Pick a model to fine-tune.
3. Create training and validation datasets.
4. Configure the fine tuning job.
5. Submit the fine tuning job.

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry
* Set an optional experiment name

**Install dependencies by running below cell. This is not an optional step if running in a new environment.**

In [ ]:
%pip install azure-ai-ml
%pip install azure-identity

%pip install mlflow
%pip install azureml-mlflow

### Create AzureML Workspace connections

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    workspace_ml_client = MLClient.from_config(credential=credential)
except:
    workspace_ml_client = MLClient(
        credential,
        subscription_id="<subscription-id>",
        resource_group_name="<resource-group-name>",
        workspace_name="<workspace-name>",
    )

# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(credential, registry_name="azureml")
registry_ml_client_meta = MLClient(credential, registry_name="azureml-meta")

# Get AzureML workspace object.
workspace = workspace_ml_client._workspaces.get(workspace_ml_client.workspace_name)
workspace._workspace_id

### 2. Pick a foundation model to fine tune

Decoder based LLM models like `llama` performs well on `text-generation` tasks, we need to finetune the model for our specific purpose in order to use it. You can browse these models in the Model Catalog in the AzureML Studio, filtering by the `text-generation` task. In this example, we use the `llama-2-7b` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

Note the model id property of the model. This will be passed as input to the fine tuning job. This is also available as the `Asset ID` field in model details page in AzureML Studio Model Catalog. 

In [ ]:
model_name = "Phi-3-mini-4k-instruct"
foundation_model = registry_ml_client.models.get(model_name, label="latest")
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for fine tuning".format(
        foundation_model.name, foundation_model.version, foundation_model.id
    )
)

In [ ]:
from azure.ai.ml.constants._common import AssetTypes
from azure.ai.ml.entities._inputs_outputs import Input
mlflow_model_llama = Input(
        type=AssetTypes.MLFLOW_MODEL, path=foundation_model.id
    )

### 3. Prepare data

Please refer to this data preparation notebook - [Data preparation](./prepare-data.ipynb)

#### Create data inputs

In [ ]:
from azure.ai.ml.entities import Data
dataset_version = "1"
dataset_dir = "samsum_dataset"
train_dataset_name = f"{dataset_dir}_train"

try:
    train_data_created = workspace_ml_client.data.get(train_dataset_name, version=dataset_version)
    print(f"Dataset {train_dataset_name} already exists")
except:
    print("creating dataset")
    train_data = Data(
        path=f"./{dataset_dir}/small_train.jsonl",
        type=AssetTypes.URI_FILE,
        description="Training dataset",
        name=train_dataset_name,
        version=dataset_version,
    )
    train_data_created = workspace_ml_client.data.create_or_update(train_data)

In [ ]:
dataset_version = "1"
validation_dataset_name = f"{dataset_dir}_validation"
try:
    validation_data_created = workspace_ml_client.data.get(validation_dataset_name, version=dataset_version)
    print(f"Dataset {validation_dataset_name} already exists")
except:
    print("creating dataset")
    validation_data = Data(
        path=f"./{dataset_dir}/small_validation.jsonl",
        type=AssetTypes.URI_FILE,
        description="Validation dataset",
        name=train_dataset_name,
        version=dataset_version,
    )
    validation_data_created = workspace_ml_client.data.create_or_update(validation_data)

In [ ]:
from azure.ai.ml.entities._inputs_outputs import Input

training_data=Input(type=train_data_created.type, path=f"azureml://locations/{workspace.location}/workspaces/{workspace._workspace_id}/data/{train_data_created.name}/versions/{train_data_created.version}")
validation_data=Input(type=validation_data_created.type, path=f"azureml://locations/{workspace.location}/workspaces/{workspace._workspace_id}/data/{validation_data_created.name}/versions/{validation_data_created.version}")

### 3. Submit the fine tuning job using the the model and data as inputs
 
Create FineTuning job using all the data that we have so far.

#### Define finetune parameters

##### There are following set of parameters that are required.

1. `model` - Base model to finetune.
2. `training_data` - Training data for finetuning the base model.
3. `task` - FineTuning task to perform. eg. TEXT_COMPLETION for text-generation/text-generation finetuning jobs.
4. `outputs`- Output registered model name.

##### Following parameters are optional:

1. `hyperparameters` - Parameters that control the FineTuning behavior at runtime.
2. `name`- FineTuning job name
3. `experiment_name` - Experiment name for FineTuning job.
4. `display_name` - FineTuning job display name.

In [ ]:
from azure.ai.ml.entities._job.finetuning.custom_model_finetuning_job import CustomModelFineTuningJob
import uuid
from azure.ai.ml._restclient.v2024_01_01_preview.models import (
    FineTuningTaskType,
)
from azure.ai.ml.entities._inputs_outputs import Output

guid = uuid.uuid4()
short_guid = str(guid)[:8]

finetuning_job = CustomModelFineTuningJob(
    task=FineTuningTaskType.TEXT_COMPLETION, # Text generation task corresponding to TEXT_COMPLETION.
    training_data=training_data,
    validation_data=validation_data,
    hyperparameters={
        "per_device_train_batch_size": "1",
        "learning_rate": "0.00002",
        "num_train_epochs": "1",
    },
    model=mlflow_model_llama,
    display_name=f"ft-job-display-name-{short_guid}",
    name=f"ft-job-{short_guid}",
    experiment_name="ft-job-finetuning-experiment",
    outputs={"registered_model": Output(type="mlflow_model", name=f"ft-job-finetune-registered-{short_guid}")},
)

In [ ]:
created_job = workspace_ml_client.jobs.create_or_update(finetuning_job)
created_job.studio_url